**Extract sentences from csv file**

In [1]:
import csv

with open('data/sentence_jpn.txt', mode='a', encoding='utf-8') as text_file:
    with open('data/cv-corpus-10/clips/test.csv', mode='r') as csv_file:
        reader = csv.reader(csv_file)

        for idx, row in enumerate(reader):
            if idx != 0:
                sentence = row[2]
                text_file.write(sentence)
                text_file.write('\n')



## Compile Kenlm

Follow the guide: https://kheafield.com/code/kenlm/

In [7]:
!python STT/data/lm/generate_lm.py \
    --input_txt data/sentence_jpn.txt \
    --output_dir scorer/ \
    --top_k 4000 \
    --kenlm_bins kenlm/build/bin/ \
    --arpa_order 5 \
    --max_arpa_memory "85%" \
    --arpa_prune "0|0|1" \
    --binary_a_bits 255 \
    --binary_q_bits 8 \
    --binary_type trie \
    --discount_fallback


Converting to lowercase and counting word occurrences ...
| |#                                               | 4394 Elapsed Time: 0:00:00

Saving top 4000 words ...

Calculating word statistics ...
  Your text file has 4395 words in total
  It has 4394 unique words
  Your top-4000 words are 91.0353 percent of all words
  Your most common word "ちょっと待ってください" occurred 2 times
  The least common word in your top-k is "シャンチの専業プロはチムから支払われる給料と対局費を主な収入としている" with 1 times
  The first word with 2 occurrences is "ちょっと待ってください" at place 0

Creating ARPA file ...
=== 1/5 Counting and sorting n-grams ===
Reading /home/nabin/office-work/Japanese-STT/scorer/lower.txt.gz
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 4395 types 4397
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:52764 2:1374936064 3:2578005248 4:412

## Package the Language Model for Deployment

In [6]:
!./native_client/generate_scorer_package \
    --checkpoint checkpoints/japanese_chekpoints \
    --lm scorer/lm.binary \
    --vocab scorer/vocab-4000.txt \
    --package scorer/kenlm.scorer \
    --default_alpha 0.931289039105002 \
    --default_beta 1.1834137581510284

4000 unique words read from vocabulary file.
Doesn't look like a character based (Bytes Are All You Need) model.
--force_bytes_output_mode was not specified, using value infered from vocabulary contents: false
Package created in scorer/kenlm.scorer.
